# Elliptic Curve Diffie-Hellman

Протокол для обмена общим секретом.

Сетап:
- $E(F_p)$
- $G \in E(F_p)$

Алиса и боб оба генерируют по случайному секретному числу:

$a, b \in [1, ord(G)-1]$

Алиса отсылает Бобу $G_a = a * G$ 

Боб отсылает Алисе $G_b = b * G$

\begin{rcases}
   a * G_b &\text{Алиса }  \\
   b * G_a &\text{Боб } 
\end{rcases} => $(a * b) * G = (b * a) * G = Sh$

In [35]:
import hashlib 

class Part:
    def __init__(self, E, G):
        self.private = randint(1, G.order()-1)
        self.E = E
        self.G = G

    def compute_public(self):
        return self.private * G

    def compute_shared(self, G_b):
        self.shared = self.private * G_b
        return self.shared

    def derive_key(self):
        self.key = hashlib.blake2b(str(self.shared).encode(), digest_size=16).digest()

    def encrypt_message(self, m: bytes):
        return bytes([x^^y for x, y in zip(m, self.key * ((len(m) + 15) // 16))])

    def decrypt_message(self, m: bytes):
        return self.encrypt_message(m).decode()
        
        
p = random_prime(2**255)
a = randint(0, p-1)
b = randint(0, p-1)
E = EllipticCurve(GF(p), [a, b])
G = E.random_element()

A = Part(E, G)
B = Part(E, G)

A_pub = A.compute_public()
B_pub = B.compute_public()

Shared_A = A.compute_shared(B_pub)
Shared_B = B.compute_shared(A_pub)
assert Shared_A == Shared_B

# Man in the middle



In [37]:
class C:
    def __init__(self, E, G):
        pass
    ...

A = Part(E, G)
B = Part(E, G)

A_pub = A.compute_public()
B_pub = B.compute_public()

Eve = C(E, G)

# do smth wiith A and B
A.compute_shared(B_pub)
B.compute_shared(A_pub)


A.derive_key()
ct = A.encrypt_message(b"Send me Hello together with our secret code")
# do smth here

B.derive_key()
m = B.decrypt_message(ct)
ct = B.encrypt_message(b"Hello, my lil potato")
# do smth here

m = A.decrypt_message(ct)
assert 'Hello' in m and 'lil potato' in m

# Гладкий порядок кривой и элемента

Допустим у нас имеется точка кривой с порядком $ord(G) = p * q$. Что произойдёт, если умножить эту точку на $p$?

In [5]:
q = 21888242871839275222246405745257275088696311157297823662689037894645226208583
E = EllipticCurve(GF(q), [0, 1])
P = e.lift_x(11287088411198481362140708319687060898747531990151311319674794660705219235317)

# your code here

In [7]:
# P = E.random_point()
# n = 79287328374952431757 * randint(0, 27797133921898830561267529521791838546)
# Q = n * P

# Небольшое введение в CRT

Вы имеете систему уравнений:

$x \equiv a_1 \pmod{p_1^{e_1}}$

$x \equiv a_2 \pmod{p_2^{e_2}}$

...

$x \equiv a_n \pmod{p_n^{e_n}}$

(Есть и более общий случай для непростых-степеней модулей но об этом не сегодня)

Тогда по Китайской Теореме об Остатках существует единственное решение этой системы:

$x = a^{*} \pmod{p_1^{e_1} * ... * p_n^{e_n}}$

В саге это можно посчитать: $x = crt([a_1, a_2, ..., a_n], [p_1^{e_1}, p_2^{e_2}, ..., p_n^{e_n}]$

In [11]:
P = E.lift_x(20784373959434472562271958623875355254756888620317246720769884886017356077580)
Q = E.lift_x(1684246270980120632629887352423092941348259671519960650766040438397726636464)
# find n?

# Aномальные кривые

Из предыдущего пункта стало ясно, что лучше использовать либо точки с большим простым порядком, либо вообще использовать кривые с простым порядком, чтобы у всех точек был один и тот же большой простой порядок. 

Аномальными называют такие кривые у которых $|E(F_p)| = p$.

Для аномальных кривых существует отображение: $E(F_p) \to F_p: \phi(P)$, причем это отображение обладает очень важным свойством: $\phi(n * P) = n * \phi(P)$ - довольно печально выглядит не правда ли?

То есть от сложной проблемы мы перешли к обычному сложению.

Отображение считается следующим образом:

Введем пару $[P, a], [Q, b]$, тогда $[P, a] + [Q, b] = [P + Q, a + b + slope(P, Q)]$ 

Где
- $Q = -P, P = O$ или $Q = O \implies slope = 0$
- $slope = \lambda$

Тогда если мы посчитаем по данному правилу $p * [P, 0] = [P, 0] + [P, 0] + ... + [P, 0] = [O, \alpha]$, $\phi(P) = \alpha$

In [14]:
def double(P, alpha):
    pass

def add(P, alpha, Q, beta):
    pass

def mul(P, n):
    pass 

In [16]:
p = 0xA15C4FB663A578D8B2496D3151A946119EE42695E18E13E90600192B1D0ABDBB6F787F90C8D102FF88E284DD4526F5F6B6C980BF88F1D0490714B67E8A2A2B77
a = 0x5E009506FCC7EFF573BC960D88638FE25E76A9B6C7CAEEA072A27DCD1FA46ABB15B7B6210CF90CABA982893EE2779669BAC06E267013486B22FF3E24ABAE2D42
b = 0x2CE7D1CA4493B0977F088F6D30D9241F8048FDEA112CC385B793BCE953998CAAE680864A7D3AA437EA3FFD1441CA3FB352B0B710BB3F053E980E503BE9A7FECE

E = EllipticCurve(GF(p), [a, b])

P = E.random_element()
Q = randint(0, p) * P

In [ ]:
# your code here